# HEALTH X - Deep Learning Models

## Contents Of Notebook

1. import statements
2. pulling raw data from data sources
3. defining functions for preprocessing 
4. CNN and CNN + LSTM model for shoulder
5. CNN and CNN + LSTM model for finger

## 1. import statements

In [50]:
import pandas as pd
import numpy as np
from numpy.random import seed
import cv2
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, MaxPooling2D, Conv2D, Flatten, Activation,ConvLSTM2D
from tensorflow import set_random_seed
set_random_seed(12567)
seed(109274)

## 2. pulling raw data from data sources

In [51]:
train_paths = pd.read_csv('train_image_paths.csv') # reading training image directories from csv file
train_paths.columns = ['image_paths'] # renaming the column

In [52]:
train_paths.head()

,image_paths
0,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
1,MURA-v1.1/train/XR_SHOULDER/patient00001/study...
2,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
3,MURA-v1.1/train/XR_SHOULDER/patient00002/study...
4,MURA-v1.1/train/XR_SHOULDER/patient00002/study...


In [26]:
valid_paths = pd.read_csv('valid_image_paths.csv') # reading validation image directories from csv file
valid_paths.columns = ['image_paths'] # renaming columns

In [27]:
valid_paths.head()

,image_paths
0,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
1,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
2,MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
3,MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...
4,MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...


## 3. defining functions for preprocessing

In [28]:
def prep_dataframe(body_part,t_v): # preparing dataframe for CNN + LSTM
    paths = t_v # pandas DataFrame containing all images
    paths.columns = ['image_paths'] # renaming column
    patient_list = []
    file_list = []
    for i in paths.image_paths: # looping over each file
        if (body_part in i): # get desired body part
            patient_num = i.split('/')[3] # get patient number
            study_num = i.split('/')[4] # get study number
            patient_study = patient_num + '/' + study_num # combining them as patient_number/study_number
            patient_list.append(patient_study) # appending patient_number/study_number to list
            file_list.append(i) # append file directory to list
    dict_dataset = {'patient':patient_list,'file_name':file_list} # combining the two lists into dictionary
    df_dataset = pd.DataFrame(dict_dataset) # converting to dataframe
    grouped = df_dataset.groupby('patient') # grouping by patient_number/study_number
    final_series = grouped['file_name'].unique() # taking unique values
    final_dataframe = pd.DataFrame({'patient':final_series.index, 'filename_array':final_series.values})
    # getting dataframe with rows in the format: patient_number/study_number  [file1,file2,file3,...]
    final_dataframe['filename_string'] = final_dataframe.filename_array.apply(','.join)
    # converting file names into string which is comma separated
    file_splits = final_dataframe['filename_string'].str.split(',', expand=True)
    # splitting on comma creates new column for each file name
    patient_files = pd.concat([final_dataframe,file_splits],axis=1)
    # concatenate splits with original dataframe
    patient_files.drop(columns=['filename_array', 'filename_string'],inplace=True)
    # removing unnecessary columns
    return(patient_files)

In [29]:
def img_cnn(df, part, img_size):
    x = []
    y = []
    for i in df.image_paths: # looping over each image
        if (part in i): # desired body part
            x.append((np.array(cv2.resize(cv2.imread(i),(img_size,img_size))))/255)
            # reading image, resizing, normalizing and appending to list
            if 'positive' in i:
                y.append(1) # assigning labels
            else:
                y.append(0)
    return(np.array(x),np.array(y))

In [30]:
def rnn_df(part):
    train = prep_dataframe(part,train_paths) # preprocessing data for CNN + LSTM
    train.set_index('patient',inplace=True)
    test = prep_dataframe(part,valid_paths) # preprocessing data for CNN + LSTM
    test.set_index('patient',inplace=True)
    if len(train.columns)>len(test.columns):
        # comparing number of columns in training and test data, setting minimum number of columns equal to 
        # the number of columns for both train and test. This is for performance evaluation
        l = len(test.columns)
        for i in range((len(train.columns)-len(test.columns))):
            test[l+i] = None
    elif len(test.columns)>len(train.columns):
        l = len(train.columns)
        for i in range((len(test.columns))-(len(train.columns))):
            train[l+i] = None   
    return(train,test)

In [31]:
def img_arr(df,img_size):
    zero_pad = [[[0 for k in range(3)] for j in range(img_size)] for i in range(img_size)]
    # creating 2D array of zeroes for missing or None images
    h = []
    for i in df.index:
        c = []
        for j in df.columns:
            if df.loc[i,j]==None:
                c.append(np.array(zero_pad))
            else:
                c.append((np.array(cv2.resize(cv2.imread(df.loc[i,j]),(img_size,img_size))))/255)
                # reading image, resizing, normalizing and appending to list
        h.append(np.array(c))
        c.clear()
        
    y = []
    for i in df.index:
        if 'positive' in i: # assigning labels
            y.append(1)
        else:
            y.append(0)
    return(np.array(h),np.array(y))
    

In [32]:
def img_rnn(part,img_size):
    train,test = rnn_df(part) # calling rnn_df internally
    x_train, y_train = img_arr(train,img_size) # calling img_arr internally
    x_test, y_test = img_arr(test,img_size) # calling img_arr internally
    return(x_train,y_train,x_test,y_test)    

## 4. SHOULDER

### CNN

In [56]:
x_train_shoulder,y_train_shoulder = img_cnn(train_paths,'SHOULDER',64)
x_test_shoulder, y_test_shoulder = img_cnn(valid_paths,'SHOULDER',64)

In [57]:
model_cnn_shoulder = Sequential()
model_cnn_shoulder.add(Conv2D(64,(3,3), input_shape=[64,64,3]))
model_cnn_shoulder.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_shoulder.add(Conv2D(32,(3,3)))
model_cnn_shoulder.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_shoulder.add(Conv2D(16,(3,3)))
model_cnn_shoulder.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_shoulder.add(Flatten())
model_cnn_shoulder.add(Dense(1000))
model_cnn_shoulder.add(Activation('relu'))
model_cnn_shoulder.add(Dense(1))
model_cnn_shoulder.add(Activation('sigmoid'))

model_cnn_shoulder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 576)               0         
__________

In [58]:
model_cnn_shoulder.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [59]:
model_cnn_shoulder.fit(x_train_shoulder,y_train_shoulder,batch_size = 100, epochs = 10)

Epoch 1/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.6979 - acc: 0.5127
Epoch 2/10
8378/8378 [==============================] - 38s 5ms/step - loss: 0.6809 - acc: 0.5654
Epoch 3/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.6607 - acc: 0.5992
Epoch 4/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.6376 - acc: 0.6364
Epoch 5/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.6169 - acc: 0.6576
Epoch 6/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.5983 - acc: 0.6799
Epoch 7/10
8378/8378 [==============================] - 41s 5ms/step - loss: 0.5770 - acc: 0.6950
Epoch 8/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.5486 - acc: 0.7205
Epoch 9/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.5219 - acc: 0.7403
Epoch 10/10
8378/8378 [==============================] - 39s 5ms/step - loss: 0.4975 - acc: 0.7527


In [60]:
model_cnn_shoulder.evaluate(x_test_shoulder,y_test_shoulder)

563/563 [==============================] - 1s 2ms/step


[0.6423304812522806, 0.6589698043005081]

### CNN + RNN (Time Distributed)

In [61]:
x_train_shoulder, y_train_shoulder, x_test_shoulder, y_test_shoulder = img_rnn('SHOULDER',64)

In [69]:
#CNN
model_mix_shoulder = Sequential()
model_mix_shoulder.add(TimeDistributed(Conv2D(64,(3,3),activation='relu'),input_shape=(x_train_shoulder.shape[1],
                                                                                        x_train_shoulder.shape[2],
                                                                                        x_train_shoulder.shape[3],
                                                                                        x_train_shoulder.shape[4])))
model_mix_shoulder.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_shoulder.add(TimeDistributed(Conv2D(32,(3,3),activation='relu')))
#model_mix_shoulder.add(TimeDistributed(Conv2D(128,(3,3),activation='relu')))
#model_mix_shoulder.add(TimeDistributed(Conv2D(56,(3,3),activation='relu')))
model_mix_shoulder.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

#model_mix_shoulder.add(TimeDistributed(Conv2D(256,(3,3),activation='relu')))
#model_mix_shoulder.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))


model_mix_shoulder.add(TimeDistributed(Flatten()))

#RNN
model_mix_shoulder.add(LSTM(100,return_sequences=False))

model_mix_shoulder.add(Dense(1))
model_mix_shoulder.add(Activation('sigmoid'))

model_mix_shoulder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_11 (TimeDis (None, 11, 62, 62, 64)    1792      
_________________________________________________________________
time_distributed_12 (TimeDis (None, 11, 31, 31, 64)    0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 11, 29, 29, 32)    18464     
_________________________________________________________________
time_distributed_14 (TimeDis (None, 11, 14, 14, 32)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 11, 6272)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               2549200   
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
__________

In [70]:
model_mix_shoulder.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [71]:
model_mix_shoulder.fit(x_train_shoulder,y_train_shoulder,batch_size = 100, epochs = 10)

Epoch 1/10
2821/2821 [==============================] - 178s 63ms/step - loss: 0.6940 - acc: 0.5055
Epoch 2/10
2821/2821 [==============================] - 196s 69ms/step - loss: 0.6933 - acc: 0.5278
Epoch 3/10
2821/2821 [==============================] - 177s 63ms/step - loss: 0.6926 - acc: 0.5197
Epoch 4/10
2821/2821 [==============================] - 1014s 360ms/step - loss: 0.6861 - acc: 0.5526
Epoch 5/10
2821/2821 [==============================] - 177s 63ms/step - loss: 0.6702 - acc: 0.6026
Epoch 6/10
2821/2821 [==============================] - 174s 62ms/step - loss: 0.6369 - acc: 0.6413
Epoch 7/10
2821/2821 [==============================] - 174s 62ms/step - loss: 0.6005 - acc: 0.6799
Epoch 8/10
2821/2821 [==============================] - 178s 63ms/step - loss: 0.5703 - acc: 0.7118
Epoch 9/10
2821/2821 [==============================] - 750s 266ms/step - loss: 0.5302 - acc: 0.7426
Epoch 10/10
2821/2821 [==============================] - 177s 63ms/step - loss: 0.4845 - acc: 0.7

In [49]:
model_mix_shoulder.evaluate(x_test_shoulder,y_test_shoulder)

194/194 [==============================] - 1s 5ms/step


[0.7715888385920181, 0.5618556701030928]

## 5. FINGER

### CNN

In [91]:
x_train_finger,y_train_finger = img_cnn(train_paths,'FINGER',64)
x_test_finger, y_test_finger = img_cnn(valid_paths,'FINGER',64)

In [92]:
np.shape(x_test_finger)

(461, 64, 64, 3)

In [93]:
model_cnn_finger = Sequential()
model_cnn_finger.add(Conv2D(64,(3,3), input_shape=[64,64,3]))
model_cnn_finger.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_finger.add(Conv2D(32,(3,3)))
model_cnn_finger.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_finger.add(Conv2D(16,(3,3)))
model_cnn_finger.add(MaxPooling2D(pool_size=(2,2)))

model_cnn_finger.add(Flatten())
model_cnn_finger.add(Dense(1000))
model_cnn_finger.add(Activation('relu'))
model_cnn_finger.add(Dense(1))
model_cnn_finger.add(Activation('sigmoid'))

model_cnn_finger.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 29, 29, 32)        18464     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 12, 12, 16)        4624      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 576)               0         
__________

In [94]:
model_cnn_finger.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [95]:
model_cnn_finger.fit(x_train_finger,y_train_finger,batch_size = 100, epochs = 7)

Epoch 1/7
5106/5106 [==============================] - 25s 5ms/step - loss: 0.6343 - acc: 0.6265
Epoch 2/7
5106/5106 [==============================] - 24s 5ms/step - loss: 0.5991 - acc: 0.6588
Epoch 3/7
5106/5106 [==============================] - 23s 5ms/step - loss: 0.5886 - acc: 0.6547
Epoch 4/7
5106/5106 [==============================] - 24s 5ms/step - loss: 0.5697 - acc: 0.6849
Epoch 5/7
5106/5106 [==============================] - 24s 5ms/step - loss: 0.5490 - acc: 0.6980
Epoch 6/7
5106/5106 [==============================] - 24s 5ms/step - loss: 0.5289 - acc: 0.7237
Epoch 7/7
5106/5106 [==============================] - 24s 5ms/step - loss: 0.5139 - acc: 0.7346


In [96]:
model_cnn_finger.evaluate(x_test_finger,y_test_finger)

461/461 [==============================] - 1s 3ms/step


[0.6308679375757106, 0.6702819960494879]

### CNN + RNN (Time distributed)

In [98]:
x_train_finger, y_train_finger, x_test_finger, y_test_finger = img_rnn('FINGER',32)

In [99]:
np.shape(x_train_finger),np.shape(y_train_finger),np.shape(x_test_finger),np.shape(y_test_finger)

((1935, 6, 32, 32, 3), (1935,), (175, 6, 32, 32, 3), (175,))

In [100]:
#CNN
model_mix_finger = Sequential()
model_mix_finger.add(TimeDistributed(Conv2D(64,(3,3),activation='relu'),input_shape=(x_train_finger.shape[1],
                                                                                        x_train_finger.shape[2],
                                                                                        x_train_finger.shape[3],
                                                                                        x_train_finger.shape[4])))
model_mix_finger.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_finger.add(TimeDistributed(Conv2D(32,(3,3),activation='relu')))
model_mix_finger.add(TimeDistributed(MaxPooling2D(pool_size=(2,2))))

model_mix_finger.add(TimeDistributed(Flatten()))

#RNN
model_mix_finger.add(LSTM(100,return_sequences=False))

model_mix_finger.add(Dense(1))
model_mix_finger.add(Activation('sigmoid'))

model_mix_finger.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_32 (TimeDis (None, 6, 30, 30, 64)     1792      
_________________________________________________________________
time_distributed_33 (TimeDis (None, 6, 15, 15, 64)     0         
_________________________________________________________________
time_distributed_34 (TimeDis (None, 6, 13, 13, 32)     18464     
_________________________________________________________________
time_distributed_35 (TimeDis (None, 6, 6, 6, 32)       0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, 6, 1152)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               501200    
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
__________

In [101]:
model_mix_finger.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [104]:
model_mix_finger.fit(x_train_finger,y_train_finger,batch_size = 100, epochs = 10)

Epoch 1/10
1935/1935 [==============================] - 12s 6ms/step - loss: 0.4782 - acc: 0.7530
Epoch 2/10
1935/1935 [==============================] - 14s 7ms/step - loss: 0.4671 - acc: 0.7587
Epoch 3/10
1935/1935 [==============================] - 16s 8ms/step - loss: 0.4695 - acc: 0.7716
Epoch 4/10
1935/1935 [==============================] - 15s 8ms/step - loss: 0.4368 - acc: 0.7788
Epoch 5/10
1935/1935 [==============================] - 13s 7ms/step - loss: 0.4266 - acc: 0.7928
Epoch 6/10
1935/1935 [==============================] - 16s 8ms/step - loss: 0.4061 - acc: 0.8072
Epoch 7/10
1935/1935 [==============================] - 13s 7ms/step - loss: 0.3661 - acc: 0.8305
Epoch 8/10
1935/1935 [==============================] - 12s 6ms/step - loss: 0.3544 - acc: 0.8439
Epoch 9/10
1935/1935 [==============================] - 12s 6ms/step - loss: 0.3196 - acc: 0.8553
Epoch 10/10
1935/1935 [==============================] - 14s 7ms/step - loss: 0.2893 - acc: 0.8739


In [105]:
model_mix_finger.evaluate(x_test_finger,y_test_finger)

175/175 [==============================] - 1s 3ms/step


[0.8087118411064148, 0.6800000017029898]